# RNN Implementation using NumPy

- Sequence input, Char-level, Batch training, Python 3

- You can train RNNs using different vector representations. (Try "glove" or "word2vec" to train the sequence of words instead of the "One-hot-vector")

- This is a slightly modified version of the original code(Karpathy).

URL

-> https://github.com/JY-Yoon

-> Original code(Karpathy) : https://gist.github.com/karpathy/d4dee566867f8291f086



In [1]:
import numpy as np

In [13]:
# load text data

# txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " # input data
txt_data = open('Quotes.txt', encoding='cp1252').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) # You can see the number of unique characters in your input data.
print("txt_data_size : ", txt_data_size)

unique characters :  56
txt_data_size :  2226


# One hot encoding

In [16]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in txt_data] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'g': 0, ':': 1, 'L': 2, '…': 3, 'u': 4, 'r': 5, 'w': 6, 'p': 7, 'j': 8, 'P': 9, 'm': 10, ',': 11, ' ': 12, 'v': 13, 'M': 14, 'A': 15, 'q': 16, 'o': 17, 'K': 18, 's': 19, 'z': 20, 'y': 21, 'I': 22, ')': 23, 'T': 24, 'D': 25, 'd': 26, '(': 27, 'i': 28, 'S': 29, '?': 30, 'b': 31, 'C': 32, 'x': 33, 'Q': 34, 'B': 35, 'N': 36, 'f': 37, '.': 38, 'R': 39, 'O': 40, 'a': 41, 'k': 42, 'h': 43, 'l': 44, '\n': 45, '’': 46, 'c': 47, 'F': 48, 'Y': 49, 'e': 50, 'n': 51, 'J': 52, 'W': 53, 't': 54, 'G': 55}
----------------------------------------------------
{0: 'g', 1: ':', 2: 'L', 3: '…', 4: 'u', 5: 'r', 6: 'w', 7: 'p', 8: 'j', 9: 'P', 10: 'm', 11: ',', 12: ' ', 13: 'v', 14: 'M', 15: 'A', 16: 'q', 17: 'o', 18: 'K', 19: 's', 20: 'z', 21: 'y', 22: 'I', 23: ')', 24: 'T', 25: 'D', 26: 'd', 27: '(', 28: 'i', 29: 'S', 30: '?', 31: 'b', 32: 'C', 33: 'x', 34: 'Q', 35: 'B', 36: 'N', 37: 'f', 38: '.', 39: 'R', 40: 'O', 41: 'a', 42: 'k', 43: 'h', 44: 'l', 45: '\n', 46: '’', 47: 'c', 48: 'F', 49: 'Y', 50: 'e', 

# Initialization

In [22]:
# hyperparameters

iteration = 500
sequence_length = 10
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 100  # size of hidden layer of neurons.  
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

# Forward propagation

In [18]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
 
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs 

# Backward propagation

In [19]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

# Training

In [23]:
data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 


for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress



iter 0, loss: 15.482792
iter 100, loss: 1.562318
iter 200, loss: 1.491933
iter 300, loss: 1.359891
iter 400, loss: 1.345444


# Prediction

In [24]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [27]:
predict('t',300) # (char, len of output)

----
 it tobk pelle toul the pon the wos we ref Louneonrin ande the wom ofkk aneestet I sines tother. Tabicere veneddkd you y nhe wsse, the wou pand on unled. Tha, (Rus. 

Duty ar cant. (an wune be teve ite  it. Sou pvithat se the pestet I ’t’o hat of Lorg toub ge the We am o


BOusw aude the ing be tht t 
----
